In [ ]:
import ibm_db


# [DataIngestion]
# # Enter the values for you database connection. This can be found in DB2's Service Credentials from the tooling.
# dsn_database = bludb
# dsn_uid      = csr11117
# dsn_pwd      = 609Np3exGwBzg1QI
# dsn_hostname = ba99a9e6-d59e-4883-8fc0-d6a8c9f7a08f.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud
# dsn_port     = 31321
# #dsn_protocol = TCPIP
# dsn_protocol= TCPIP
# dsn_driver   = IBM DB2 ODBC DRIVER
# #delimiter of  input file
# delimiter = ,
# db_type = db2


conn_str = (
    "DRIVER={IBM DB2 ODBC DRIVER};"
    "DATABASE=bludb;"
    "HOSTNAME=ba99a9e6-d59e-4883-8fc0-d6a8c9f7a08f.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud;"
    "PORT=31321;"
    "PROTOCOL=TCPIP;"
    "UID=csr11117;"
    "PWD=609Np3exGwBzg1QI;"
    "SECURITY=SSL;"
)

# conn_str = (
#     "DATABASE=gosales;"
#     "HOSTNAME=useast.services.cloud.techzone.ibm.com;"
#     "PORT=22879;"
#     "PROTOCOL=TCPIP;"
#     "UID=DB2INST1;"
#     "PWD=db2inst1;"
# )

# conn_str = (
#     "DRIVER={IBM DB2 ODBC DRIVER};"
#     "DATABASE=bludb;"
#     "HOSTNAME=54a2f15b-5c0f-46df-8954-7e38e612c2bd.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud;"
#     "PORT=32733;"
#     "PROTOCOL=TCPIP;"
#     "UID=dql90486;"
#     "PWD=knuCFLOLeFD1qXzX;"
#     "SECURITY=SSL;"
# )

conn = ibm_db.connect(conn_str, "", "")
if conn:
    print("Connected to the database.")
else:
    print("Connection failed.")



# create_db_sql = "CREATE DATABASE owais_database"
# ibm_db.exec_immediate(admin_conn, create_db_sql)



# conn_str = (
#     "DATABASE=gosales;"
#     "HOSTNAME=useast.services.cloud.techzone.ibm.com;"
#     "PORT=22879;"
#     "PROTOCOL=TCPIP;"
#     "UID=DB2INST1;"
#     "PWD=db2inst1;"
# )
# create_schema_sql = "CREATE SCHEMA test_owais"
# ibm_db.exec_immediate(conn, create_schema_sql)



create_table_sql = """
CREATE TABLE bludb.owais_table (
    id INT PRIMARY KEY NOT NULL,
    name VARCHAR(100),
    age INT
)
"""
ibm_db.exec_immediate(conn, create_table_sql)


insert_data_sql = """
INSERT INTO bludb.owais_table (id, name, age) 
VALUES (?, ?, ?)
"""
stmt = ibm_db.prepare(conn, insert_data_sql)
data = [
    (10, 'Alice', 30),
    (20, 'Bob', 25),
    (30, 'Charlie', 51)
]
for row in data:
    ibm_db.execute(stmt, row)


select_sql = "SELECT * FROM bludb.owais_table"
stmt = ibm_db.exec_immediate(conn, select_sql)

result = []
row = ibm_db.fetch_assoc(stmt)
while row:
    result.append(row)
    row = ibm_db.fetch_assoc(stmt)

for row in result:
    print(row)


print("Schema creation and Table insertion successful")


In [ ]:
drop_table_sql = "DROP TABLE bludb.owais_table"
ibm_db.exec_immediate(conn, drop_table_sql)

# drop_schema_sql = "DROP SCHEMA test_owais RESTRICT"
# ibm_db.exec_immediate(conn, drop_schema_sql)

In [4]:
import pandas as pd
import ibm_db
import configparser



# User expertConfig
dsn_database = ""
dsn_uid = ""
dsn_pwd = ""
dsn_hostname = ""
dsn_port = ""
dsn_protocol = "TCPIP"
dsn_driver = "{IBM DB2 ODBC DRIVER};"


def db2_connector():
    dsn = ("DRIVER={{IBM DB2 ODBC DRIVER}};" "DATABASE={0};" "HOSTNAME={1};" "PORT={2};" "PROTOCOL=TCPIP;" "UID={3};" "PWD={4};SECURITY=SSL").format(dsn_database, dsn_hostname, dsn_port, dsn_uid, dsn_pwd)
    options = {ibm_db.SQL_ATTR_AUTOCOMMIT: ibm_db.SQL_AUTOCOMMIT_ON}
    conn = ibm_db.connect(dsn, "", "", options)
    return conn

def db2_connection_close(conn):
    ibm_db.close(conn)


def db2_connectorWithSchema(schema_name):
    schema_name =schema_name.upper()
    dsn = ("DRIVER={{IBM DB2 ODBC DRIVER}};"
           "DATABASE={0};"
           "HOSTNAME={1};"
           "PORT={2};"
           "PROTOCOL=TCPIP;"
           "UID={3};"
           "PWD={4};"
           "SECURITY=SSL;"
           "CURRENTSCHEMA={5}").format(dsn_database, dsn_hostname, dsn_port, dsn_uid, dsn_pwd, schema_name)

    options = {ibm_db.SQL_ATTR_AUTOCOMMIT: ibm_db.SQL_AUTOCOMMIT_ON}
    conn = ibm_db.connect(dsn, "", "", options)

    return conn


In [5]:
import pandas as pd
import ibm_db
import configparser
# import db2_connector as dbcon

try:
    drop_table_sql = "DROP TABLE TEST.CONTRACTS_DATABASE_OWAIS"
    ibm_db.exec_immediate(conn, drop_table_sql)
except:
    pass


delimiter = ","

try:
    conn = db2_connector()
except Exception as e:
    print(f"Error connecting to the database: {e}")
    exit(1)

def load_data_into_db(filename, table_name, schema_name, delimiter=delimiter):
    try:
        df = pd.read_csv(filename, sep=delimiter)
        df = df[['id','function','supplier','services','effective_date','keywords',\
                                                   'document_type','expiration_date','region','tcv','term_type','title','status',\
                                                  'functions','countries','regions']] #'created_by','keyword'
        df = df.head()    
    except Exception as e:
        print(f"Error reading CSV file: {e}")
        return
    
    def table_exists(conn, table_name, schema_name):
        sql = """
        SELECT 1 FROM SYSCAT.TABLES
        WHERE TABSCHEMA = ? AND TABNAME = ?
        FETCH FIRST ROW ONLY
        """
        print(f"SQL Query: {sql}")
        print(f"Schema Name: {schema_name.upper()}")
        print(f"Table Name: {table_name.upper()}")
        
        try:
            stmt = ibm_db.prepare(conn, sql)
            print(f"Statement prepared: {stmt}")
            
            ibm_db.bind_param(stmt, 1, schema_name.upper())
            ibm_db.bind_param(stmt, 2, table_name.upper())
            print("Parameters bound to the statement.")
            
            ibm_db.execute(stmt)
            print(f"Statement executed: {stmt}")
            
            result = ibm_db.fetch_assoc(stmt)
            print(f"Query Result: {result}")
            
            return result
        except Exception as e:
            print(f"Error checking if table exists: {e}")
            return False
    
    columns_schema = [(column, 'INT' if dtype == 'int64' or dtype == 'int32' else 'FLOAT' if dtype == 'float64' or 
                       dtype == 'float32' else 'VARCHAR(1000)') for column, dtype in zip(df.columns, df.dtypes)]
    print(table_exists(conn, table_name, schema_name))
    # if not table_exists(conn, table_name, schema_name):
    #     create_table_sql = "CREATE TABLE {}.{} (".format(schema_name, table_name)
        
    #     for column, data_type in columns_schema:
    #         create_table_sql += "{} {}, ".format(column, data_type)
    #     create_table_sql = create_table_sql[:-2] + ")"
    #     try:
    #         print(create_table_sql)
    #         stmt = ibm_db.exec_immediate(conn, create_table_sql)
    #     except Exception as e:
    #         print(f"Error creating table: {e}")
    #         return
    if not table_exists(conn, table_name, schema_name):
    # Create table SQL statement
        create_table_sql = f"CREATE TABLE {schema_name}.{table_name} ("
        create_table_sql += ", ".join(f"{column} {data_type}" for column, data_type in columns_schema)
        create_table_sql += ")"
        
        try:
            print(f"Creating table with SQL: {create_table_sql}")
            stmt = ibm_db.exec_immediate(conn, create_table_sql)
            print("Table created successfully.")
        except Exception as e:
            print(f"Error creating table: {e}")
        columns = ','.join(df.columns)
        placeholders = ','.join(['?'] * len(df.columns))
        insertSQL = f'INSERT INTO {schema_name}.{table_name} ({columns}) VALUES({placeholders})'
        
        try:
            stmt = ibm_db.prepare(conn, insertSQL)
        except Exception as e:
            print(f"Error preparing SQL statement: {e}")
            return
    
        try:
            for n in range(len(df)):
                for i, col in enumerate(df.columns, start=1):
                    value = df.at[n, col]
                    if isinstance(value, int):
                        data_type = ibm_db.SQL_INTEGER
                    elif isinstance(value, float):
                        data_type = ibm_db.SQL_DECIMAL
                    else:
                        data_type = ibm_db.SQL_VARCHAR
                    ibm_db.bind_param(stmt, i, str(value), ibm_db.SQL_PARAM_INPUT, data_type)
                ibm_db.execute(stmt)
        except Exception as e:
            print(f"Error inserting data: {e}")
            return

    try:
        ibm_db.close(conn)
    except Exception as e:
        print(f"Error closing the connection: {e}")

    return "Data Ingested into table: {}.{}".format(schema_name, table_name)

In [ ]:
filename = '/home/askyourcorpus/owais_querycraft_experiments/QueryCraft-fork/synthetic_data.csv'
table_name = 'contracts_database_owais'
schema_name = 'test'


print(load_data_into_db(filename, table_name, schema_name))

In [ ]:
drop_table_sql = "DROP TABLE TEST.CONTRACTS_DATABASE_OWAIS"
ibm_db.exec_immediate(conn, drop_table_sql)

In [ ]:
df = pd.read_csv(filename, sep=delimiter)[['id','function','supplier','services','effective_date','keywords',\
                                           'document_type','expiration_date',]]
df = df.head()
df

In [ ]:
df = pd.read_csv(filename, sep=delimiter)#[['id','function','supplier']]
df